###### Convert PyTorch (.pt/.pth) to onnx 

In [1]:
import io
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
import numpy as np
import torch.onnx


In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3)
        # nn.init.xavier_normal_(self.conv1.weight)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3)
        # nn.init.xavier_normal_(self.conv2.weight)
        self.drop1 = nn.Dropout2d(0.3)
        self.drop2 = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(800, 64)
        # nn.init.xavier_normal_(self.fc1.weight)
        self.fc2 = nn.Linear(64, 302)
        # nn.init.xavier_normal_(self.fc2.weight)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(F.relu(self.conv2(x)) , 2)
        x= self.drop1(x)
        x= self.flatten(x)
        x= self.fc1(x)
        x= self.drop2(x)
        x= self.fc2(x)
        # return F.log_softmax(x, dim=1)
        return x
        

In [3]:
mdl_name = "CombinedMNIST_ptrchmdl_27July"
#Model class instance
model = Net()

#Saved PyTorch model path
# model_path = "/Users/kpangalu/Downloads/Pytorch_Mnist_28Jul/chkpoint/chkpoint/July27_acc_0.9994_iter_265_model.pth"
model_path = "/Users/kpangalu/Downloads/Pytorch_Mnist_28Jul/chkpoint_staticnormal/Aug19_accTrain_0.9990_accTest_0.9988_iter_415_model.pt"
batch_size = 1

#Load the weights from a file (.pth usually)
mdl_wts_fle = torch.load(model_path)

#Load weights now into a model net architecture defined by our class
model.load_state_dict(mdl_wts_fle)
model_eval = model.eval()
display(model_eval)

#create the right input shape of an image 
#dummy_input = torch.randn(sample_batch_size, channel, height, width)
dummy_input = torch.randn(1,1,14,24)

# onnx_path = "/Users/kpangalu/Downloads/Pytorch_Mnist_28Jul/" + mdl_name +'.onnx'
onnx_path = "/Users/kpangalu/Downloads/Pytorch_Mnist_28Jul/chkpoint_staticnormal/Aug19_accTrain_0.9990_accTest_0.9988_iter_415_model.onnx"
torch.onnx.export(model, dummy_input, onnx_path)

Net(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
  (drop1): Dropout2d(p=0.3, inplace=False)
  (drop2): Dropout(p=0.5, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=800, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=302, bias=True)
)